In [178]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

LC = pd.read_csv("C:\\Users\\skyamz\\Desktop\\ppdai_3_213\\LC.csv") 
LP = pd.read_csv("C:\\Users\\skyamz\\Desktop\\ppdai_3_213\\LP.csv")

In [179]:
#把日期都转换程日期格式，方便计算
LP['到期日期'] = pd.to_datetime(LP['到期日期'])
LP['还款日期'] = LP['还款日期'].replace('\\N','2017-02-22')#将还款日期不明确的标的，还款日期统一设置为记录日期，即2017-02-22                
LP['还款日期'] = pd.to_datetime(LP['还款日期'])
LC['orn_rank'] = LC['初始评级']#为了防止中文乱码的问题，把’初始评级‘改成’orn_rank‘

In [180]:
#计算逾期，并将其转换成int格式，为了以后计算方便
LP['day_diff'] =  LP['还款日期'] - LP['到期日期']
LP['day_diff'] = LP['day_diff'].astype(np.str) #先转换成string格式，然后利用map函数来逐一变成数字
Tran_Date = lambda day_diff: int(day_diff.split('days',1)[0])
LP['day_diff']= LP['day_diff'].map(Tran_Date)

In [181]:
LP02 = LP[(LP['还款状态']==2)|(LP['还款状态']==0)]#选择已经逾期的标的和未还款的标的

In [182]:
LP02 = LP02[(LP02['day_diff']<=90)&(LP02['day_diff']>=1)] #催收回款的定义为逾期 90 天之内的逾期还款                                        

In [183]:
LP02 = pd.merge(LC,LP02,on='ListingId')#内联LC与LP

In [219]:
def cla(n,lim):  # map function  n是年龄值，lim是每个年龄段的长度
    return'[%.f,%.f)'%(lim*(n//lim),lim*(n//lim)+lim) 
LP02['借款金额范围'] = pd.Series([cla(s,25000) for s in LP02['借款金额']])

In [220]:
LP02['借款金额范围'] 

0             [0,25000)
1             [0,25000)
2             [0,25000)
3             [0,25000)
4             [0,25000)
5             [0,25000)
6             [0,25000)
7         [25000,50000)
8             [0,25000)
9         [25000,50000)
10        [25000,50000)
11            [0,25000)
12            [0,25000)
13            [0,25000)
14            [0,25000)
15            [0,25000)
16            [0,25000)
17            [0,25000)
18            [0,25000)
19            [0,25000)
20            [0,25000)
21            [0,25000)
22            [0,25000)
23            [0,25000)
24            [0,25000)
25            [0,25000)
26            [0,25000)
27            [0,25000)
28            [0,25000)
29            [0,25000)
              ...      
238778        [0,25000)
238779        [0,25000)
238780        [0,25000)
238781        [0,25000)
238782        [0,25000)
238783        [0,25000)
238784        [0,25000)
238785        [0,25000)
238786        [0,25000)
238787        [0,25000)
238788        [0

In [221]:
#根据借款金额范围和逾期，对逾期和未还的标的进行分类
lable = ['借款金额范围','day_diff']
level = pd.DataFrame(LP02.pivot_table(index=lable,columns=['还款状态'],values=['应还本金'],aggfunc=np.sum))

In [222]:
level

应还本金              
还款状态                     0             2
借款金额范围         day_diff                         
[0,25000)      1         364313.10  2.689282e+07
               2         368340.81  1.022742e+07
               3         338899.34  6.571977e+06
               4         267754.33  1.271998e+07
               5         247339.89  5.999780e+06
               6         251317.98  3.804756e+06
               7         173413.00  2.768927e+06
               8         183081.73  2.001269e+06
               9         176334.81  1.486354e+06
               10        262599.13  1.215959e+06
               11        147269.25  9.805615e+05
               12        163017.88  7.718444e+05
               13        127617.36  6.581260e+05
               14        134153.61  5.665485e+05
               15        109256.04  5.118783e+05
               16        137045.34  9.288442e+05
               17        126989.77  6.899855e+05
               18        142379.72  4.980180e+05
               19        109919.19  3.819707e+05
               20        129096.48  3.469657e+05
               21        115263.12  2.774370e+05
               22         81505.70  2.276070e+05
               23        105947.58  2.251210e+05
               24        127016.07  1.995578e+05
               25        138153.24  1.789611e+05
               26        101722.59  1.957260e+05
               27        111790.56  1.619717e+05
               28        112771.39  1.526691e+05
               29        106245.03  1.202507e+05
               30        110356.23  1.405432e+05
...                            ...           ...
[75000,100000) 5               NaN  4.389880e+04
               6               NaN  4.859941e+04
               7               NaN  9.426188e+04
               8               NaN  1.341256e+04
               9               NaN  1.787778e+04
               10              NaN  1.629598e+04
               11              NaN  1.141904e+04
               12              NaN  1.150438e+04
               14              NaN  1.152859e+04
               15              NaN  9.731290e+03
               17              NaN  1.651464e+04
               18         15469.67  7.018250e+03
               20              NaN  6.778230e+03
               21              NaN  4.634340e+03
               22              NaN  1.009854e+04
               24              NaN  1.617155e+04
               25              NaN  6.857310e+03
               28              NaN  9.607690e+03
               29              NaN  9.806660e+03
               30              NaN  4.344210e+03
               33              NaN  8.916360e+03
               34              NaN  4.682500e+03
               35              NaN  6.874750e+03
               36              NaN  4.792400e+03
               37              NaN  9.175980e+03
               38              NaN  5.444580e+03
               45              NaN  4.890870e+03
               63          5508.13           NaN
               64          5140.90           NaN
               69              NaN  5.381800e+03

[439 rows x 2 columns]

In [223]:
level.columns= ['0','2']
level = level.fillna(0)
#计算催收回款率（back_rate） = 催收回本金(2)/所有逾期本金(total)
level['total'] = level['0'] + level['2']
level['back_rate'] = level['2']/level['total']
level['back_rate'] = level['back_rate'].astype(np.str)#将back_rate变为str，方便输出
level = level.reset_index()#将level标中的标签变换为列名

In [224]:
level

,借款金额范围,day_diff,0,2,total,back_rate
0,"[0,25000)",1,364313.10,2.689282e+07,2.725713e+07,0.986634208452
1,"[0,25000)",2,368340.81,1.022742e+07,1.059576e+07,0.965236954548
2,"[0,25000)",3,338899.34,6.571977e+06,6.910877e+06,0.950961455001
3,"[0,25000)",4,267754.33,1.271998e+07,1.298774e+07,0.979384067559
4,"[0,25000)",5,247339.89,5.999780e+06,6.247120e+06,0.960407371163
5,"[0,25000)",6,251317.98,3.804756e+06,4.056074e+06,0.938039111003
6,"[0,25000)",7,173413.00,2.768927e+06,2.942340e+06,0.941062888708
7,"[0,25000)",8,183081.73,2.001269e+06,2.184351e+06,0.916184846248
8,"[0,25000)",9,176334.81,1.486354e+06,1.662688e+06,0.893945971731
9,"[0,25000)",10,262599.13,1.215959e+06,1.478558e+06,0.822395125056


In [225]:
rank  = list(level['借款金额范围'].unique())#取初始评级
day_diff,back_rate = [],[]
#根据不同的初始评级，将逾期和催收回款率存入list中，方便画图
for i in rank:
        temp = level[level['借款金额范围']==i]
        day_diff.append(list(temp['day_diff']))
        back_rate.append(list(temp['back_rate']))

In [226]:
rank

['[0,25000)',
 '[100000,125000)',
 '[125000,150000)',
 '[150000,175000)',
 '[175000,200000)',
 '[200000,225000)',
 '[225000,250000)',
 '[25000,50000)',
 '[250000,275000)',
 '[275000,300000)',
 '[300000,325000)',
 '[325000,350000)',
 '[350000,375000)',
 '[375000,400000)',
 '[400000,425000)',
 '[450000,475000)',
 '[475000,500000)',
 '[50000,75000)',
 '[500000,525000)',
 '[75000,100000)']

In [227]:
len(day_diff)

20

In [228]:
back_rate

[['0.986634208452',
  '0.965236954548',
  '0.950961455001',
  '0.979384067559',
  '0.960407371163',
  '0.938039111003',
  '0.941062888708',
  '0.916184846248',
  '0.893945971731',
  '0.822395125056',
  '0.869422556063',
  '0.825623650918',
  '0.837583909239',
  '0.808544021531',
  '0.824102390995',
  '0.871426319386',
  '0.844561061872',
  '0.77766986915',
  '0.776536987365',
  '0.728824308524',
  '0.70648563624',
  '0.736323720067',
  '0.67998298117',
  '0.611064834363',
  '0.564342433133',
  '0.658016276626',
  '0.59165100527',
  '0.575153775522',
  '0.530918280243',
  '0.560157597776',
  '0.578346195781',
  '0.589913113548',
  '0.53925668015',
  '0.525248145827',
  '0.415608316787',
  '0.413278312658',
  '0.371758948428',
  '0.431344354207',
  '0.380570620395',
  '0.448991369085',
  '0.276172812309',
  '0.358350469108',
  '0.373626649129',
  '0.448995469344',
  '0.403459815097',
  '0.397407870995',
  '0.338506998182',
  '0.218643665626',
  '0.2452190372',
  '0.394057083756',
  '0.25

In [229]:
#绘制初始评级对催收回款率影响的折线图
fig = plt.figure(figsize = (20,10))#设置大小
plt.title('Loan amount influnence')#标题
plt.xlabel('day_diff')#X轴
plt.ylabel('back_rate')#Y轴
#分别对应['A','B','C','D','E','F'] 的颜色
colors = ['red','blue','green','orange','black','purple']
#对于每个初始评级，绘制它们对应的催收回款率
for i in range(0,len(day_diff)):
        plt.plot(day_diff[i],back_rate[i],'',label = rank[i])
        plt.legend(loc='lower left')#标签位置

In [230]:
plt.show()